In [1]:
import torch
import pandas as pd
import random
import json
from tqdm.notebook import tqdm
import sys
sys.path.append('../')
from util import load_node_csv

class Args:
    pass

args = Args()
args.gpu = 'cuda:2'
args.data_path = "../data/kobaco.csv"
args.num_iters = 10000
args.batch_size = 512
args.lambda_val = 1e-6

In [2]:
df = pd.read_csv(args.data_path)
user_mapping = load_node_csv(args.data_path, index_col='user_id')
item_mapping = load_node_csv(args.data_path, index_col='item_id')
inv_user_mapping = {v:k for k,v in user_mapping.items()}
inv_item_mapping = {v:k for k,v in item_mapping.items()}

In [3]:
total_items = df.item_id.tolist()
temp_df = df.groupby('user_id')['item_id'].apply(list).reset_index()

train_data = []
test_data = []

train_edge_index = [[], []]
test_edge_index = [[], []]

random.seed(1004)
for idx, row in tqdm(temp_df.iterrows(), total=len(temp_df)):
    user, items = row['user_id'], row['item_id']

    question = f'질문: 사용자 {user}의 TV 프로그램 시청 기록을 고려했을 때, 타겟 TV 프로그램을 사용자가 선호할지 판단해주세요. 반드시 "예" 또는 "아니"로만 대답해야합니다.\n답변: '
    negative_samples = list(set(total_items) - set(items))
    
    # 최종 성능 체크를 위한 데이터셋
    test_target = random.choice(items)
    test_interacted = list(set(items) - set(test_target))
    
    # 학습을 위한 데이터셋
    train_target = random.sample(test_interacted, 4)
    train_interacted = list(set(test_interacted) - set(train_target))

    # Train edge index
    train_edge_index[0].extend([user_mapping[user]]*len(train_interacted))
    train_edge_index[1].extend([item_mapping[item] for item in train_interacted])

    # Test edge index
    test_edge_index[0].extend([user_mapping[user]]*len(test_interacted))
    test_edge_index[1].extend([item_mapping[item] for item in test_interacted])

    for target in train_target:
        # train positive sample
        train_data.append({
            'question':question,
            'answer': '예',
            'user_id':user,
            'interacted_items': random.sample(train_interacted, 30),
            'target_item': target,
        })
        
        # train negative sample
        train_data.append({
            'question':question,
            'answer': '아니',
            'user_id':user,
            'interacted_items': random.sample(train_interacted, 30),
            'target_item': random.choice(negative_samples),
        })

    # test positive sample
    test_data.append({
        'question':question,
        'answer': '예',
        'user_id':user,
        'interacted_items': random.sample(test_interacted, 30),
        'target_item': test_target,
    })
    
    # test negative sample
    test_data.append({
        'question':question,
        'answer': '아니',
        'user_id':user,
        'interacted_items': random.sample(test_interacted, 30),
        'target_item': random.choice(negative_samples),
    })

  0%|          | 0/2833 [00:00<?, ?it/s]

In [4]:
with open("../data/train.jsonl", "w") as f:
    for data in train_data:
        f.write( json.dumps(data, ensure_ascii=False) + "\n" )

with open("../data/test.jsonl", "w") as f:
    for data in test_data:
        f.write( json.dumps(data, ensure_ascii=False) + "\n" )

torch.save(torch.Tensor(train_edge_index), '../data/train_edge_index.pt')
torch.save(torch.Tensor(test_edge_index), '../data/test_edge_index.pt')

In [5]:
data = train_data[0]
user_id = data['user_id']
prompt =f'사용자 {user_id}의 TV 프로그램 시청 기록:\n'
for idx, item in enumerate(data['iteracted_items']):
    prompt += f'{idx}. {item}\n'
prompt+='\n타겟 TV 프로그램:\n* ' + data['target_item'] + '\n\n'
prompt+= ''+data['question'] + data['answer']
print(prompt)

사용자 2의 TV 프로그램 시청 기록:
0. 전기현의 씨네뮤직
1. 식객 허영만의 백반기행
2. 데블스 오운 1부
3. 내부자들
4. 월수금화목토
5. 코로나19 통합뉴스룸 KBS 뉴스 12
6. 구필수는 없다
7. 뮤직뱅크
8. 이슈 픽 쌤과 함께
9. 삼남매가 용감하게
10. 지금부터, 쇼타임!
11. 놀라운 토요일
12. KBS 뉴스특보
13. 동상이몽2 너는 내 운명
14. KBS 뉴스 12
15. 한국영화특선 <두 얼굴의 여친>
16. 제로섬게임
17. 우리들의 블루스
18. 뜻밖의 여정
19. 무엇이든 물어보세요
20. MBC 희망프로젝트 어른이 되던 날
21. 스포츠 투나잇
22. MBC 네트워크특선
23. 일당백집사 
24. 다큐플렉스 
25. 출발! 비디오 여행
26. 2022 평창 대관령음악제
27. 독립영화관 <고백>
28. 홍진경의 영화로운 덕후생활
29. 2시 뉴스 외전

타겟 TV 프로그램:
* MBC 뉴스투데이 2부

질문: 사용자 2의 TV 프로그램 시청 기록을 고려했을 때, 타겟 TV 프로그램을 사용자가 선호할지 판단해주세요. 반드시 "예" 또는 "아니"로만 대답해야합니다.
답변: 예
